In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = 20, 10

from instamatic.processing.indexer import *
from instamatic.tools import get_files
import glob
import h5py

In [ ]:
from collections import OrderedDict, defaultdict
d = defaultdict(OrderedDict)

d["title"] = "indexing"

d["experiment"]["pixelsize"] = 0.00411
d["projections"]["dmin"] = 1.0
d["projections"]["dmax"] = 10.0
d["projections"]["thickness"] = 400
d["projections"]["wavelength"] = 0.0251

d["cell"]["params"] = (24.2576, )
d["cell"]["name"] = "NaY"
d["cell"]["spgr"] = "Fd-3m"
d["cell"]["composition"] = "Si96 O192"

d["data"]["files"] = "stream.txt"

d["data"]["csv_out"] = "results.ycsv"
d["data"]["drc_out"] = "indexing"

yaml_ordered_dump(d, "indexing.inp", default_flow_style=False)

In [ ]:
d = yaml_ordered_load("indexing.inp")

pixelsize = d["experiment"]["pixelsize"]
files = d["data"]["files"]

method = "powell"
radius = 3
nsolutions = 25
filter1d = False
nprojs = 100
vary_scale = True
vary_center = True

if isinstance(d["cell"], (tuple, list)):
    pixelsize = d["experiment"]["pixelsize"]
    indexer = IndexerMulti.from_cells(d["cell"], pixelsize=pixelsize, **d["projections"])
else:
    projector = Projector.from_parameters(**dict(d["cell"].items() + d["projections"].items()))
    indexer = Indexer.from_projector(projector, pixelsize=d["experiment"]["pixelsize"])

In [ ]:
all_refined = {}
all_results = {}

fns = get_files(files)
# fns = glob.glob("./processed/*.h5")

if True:
    for i, fn in enumerate(fns):
        print fn
            
        f = h5py.File(fn)

        data = np.array(f["data"])
        rawdata = np.array(f["rawdata"])
        
        data = np.clip(data, 0, data.max())
        
        center = np.array(f["peakinfo/beam_center"])
        
        ax1 = plt.subplot(121, aspect="equal")
        ax2 = plt.subplot(122, aspect="equal")
        
        results = indexer.index(data, center, nsolutions=nsolutions, 
                                filter1d=filter1d, nprojs=nprojs)
        all_results[fn] = results[0]
        indexer.plot(data, results[0], vmin=0, vmax=250, ax=ax1)
        
        refined = indexer.refine_all(data, results, sort=True, 
                                     method=method, vary_scale=vary_scale, vary_center=vary_center)
        best = refined[0]
        all_refined[fn] = best
        
        indexer.plot(rawdata, best, vmin=0, vmax=250, ax=ax2)
        plt.show()
    
        f.close()
        # if i > 10:break


In [ ]:
%load_ext line_profiler
%lprun -f indexer.find_orientation indexer.index(img, center, nsolutions=25)

In [ ]:
df = results2df(all_refined)
df = df.sort_index()
# df = df[df.score > 150]
print len(df)
plt.plot(df.alpha.values, marker="o", lw=0, label="alpha")
plt.plot(df.beta.values, marker="o", lw=0, label="beta")
plt.plot(df.gamma.values, marker="o", lw=0, label="gamma")
# plt.ylim(0, np.pi)
plt.legend()
df

In [ ]:
df = results2df(all_results)
df = df.sort_index()
# df = df[df.score > 150]
print len(df)
plt.plot(df.alpha.values, marker="o", lw=0, label="alpha")
plt.plot(df.beta.values, marker="o", lw=0, label="beta")
plt.plot(df.gamma.values, marker="o", lw=0, label="gamma")
# plt.ylim(0, np.pi)
plt.legend()

In [ ]:
number,alpha,beta,gamma,center_x,center_y,scale = 803,0.978,0.4169,0.9026,1065,1011,643.087
proj = indexer.projector.get_projection(alpha, beta, gamma)
hkl = proj[:,0:3]
pks = proj[:,3:5]
factor = proj[:,5]